In [2]:
COLAB = True

if COLAB:
    # make a connection with my google drive
    from google.colab import drive
    drive.mount('/content/drive/')

    # move to project file
    %cd drive/MyDrive/python_project/BERTforNER/

    # install packages
    !pip install -r requirements.txt


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/python_project/BERTforNER/'
/content/drive/MyDrive/python_project/BERTforNER
     |████████████████████████████████| 748.8 MB 19 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu111 requires torch==1.9.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.6.0 which is incompatible.


In [3]:
import datasets
from transformers import BertTokenizer, BertForTokenClassification, AdamW
import torch
from torch import tensor
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import os
import pickle
from seqeval.metrics import f1_score

In [4]:
# load wikiann dataset and select 'en' language 
data = datasets.load_dataset('wikiann', 'en').shuffle(seed=11)
train_dataset, validation_dataset, test_dataset = data['train'], data['validation'], data['test']

# tags: person(PER), organisation(ORG), location(LOC) in the IOB2 format
ner_tags = {0:"O", 1:"B-PER", 2:"I-PER", 3:"B-ORG", 4:"I-ORG", 5:"B-LOC", 6:"I-LOC"}

Reusing dataset wikiann (/root/.cache/huggingface/datasets/wikiann/en/1.1.0/a15e045b963d8f93f344ace01804ad38ed4b750c7c53f95f401267781ad700ca)


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(ner_tags)).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [6]:
class NER_Dataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.max_len = 128
        
    def __len__(self):
        return len(self.dataset)        
    
    def __getitem__(self, index):
        self.tokens = self.dataset[index]['tokens']
        self.labels = self.dataset[index]['ner_tags']

        # tokenize the token
        encoded_tokens = tokenizer(self.tokens, add_special_tokens=False)
        
        # determine the index of the last token we use 
        # such that sum(num_of_tokens) <= (max_len - 2) (2 are special tokens)
        num_of_tokens = [len(x) for x in encoded_tokens['input_ids']]
        end_token_index = self.less_than_max_len(num_of_tokens)
        
        # combine ids by tokenized tokens
        input_ids = self.combine_ids(encoded_tokens['input_ids'], end_token_index)
        token_type_ids = self.combine_ids(encoded_tokens['token_type_ids'], end_token_index)
        attention_mask = self.combine_ids(encoded_tokens['attention_mask'], end_token_index)
        
        # propagate labels by the length of every tokenized tokens
        labels = self.propagate_labels(encoded_tokens['input_ids'], self.labels, end_token_index)

        # need for training
        padded_input_ids, padded_token_type_ids, padded_attention_mask, padded_labels = self.padding(input_ids, token_type_ids, attention_mask, labels)

        ''' num_of_tokens is used for rebuild original labels, which had been propagated

            e.g. 'tokens' = ['R.H.','Saunders','(','St.','Lawrence','River',')','(','968','MW',')']

                 'encoded_tokens' = {'input_ids': [[1054, 1012, 1044, 1012], [15247], [1006], [2358, 1012], [5623], [2314], [1007], [1006], [5986, 2620], [12464], [1007]], 
                                     'token_type_ids': [[0, 0, 0, 0], [0], [0], [0, 0], [0], [0], [0], [0], [0, 0], [0], [0]], 
                                     'attention_mask': [[1, 1, 1, 1], [1], [1], [1, 1], [1], [1], [1], [1], [1, 1], [1], [1]]}

                 num_of_tokens = [4, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1] + [0]* (self.max_len - len(num_of_tokens)) '''

        num_of_tokens = [len(x) for x in encoded_tokens['input_ids']][:(end_token_index+1)]
        num_of_tokens = num_of_tokens + [0]* (self.max_len - len(num_of_tokens)) if len(num_of_tokens) <= self.max_len else num_of_tokens[:self.max_len]
        return tensor(padded_input_ids), tensor(padded_token_type_ids), tensor(padded_attention_mask), tensor(padded_labels), tensor(num_of_tokens)

    
    def less_than_max_len(self, a_list):
        summation = 0
        for index in range(len(a_list)):
            summation += a_list[index]
            if summation > (self.max_len-2):
                return (index-1)
                break
            if summation == (self.max_len-2):
                return (index)
                break
        return (index)
    
    def combine_ids(self, ids, end_token_index):
        comb_ids = []
        for x in ids[:(end_token_index+1)]:
            comb_ids += x
        return comb_ids

    def propagate_labels(self, ids, labels, end_token_index):
        PROP_labels = []
        for i in range(0, (end_token_index+1)):
            PROP_labels += [labels[i]]* len(ids[i])
        PROP_labels = PROP_labels
        return PROP_labels

    def padding(self, input_ids, token_type_ids, attention_mask, labels):
        padding_len = max(0, (self.max_len - len(input_ids) - 2))
        padded_input_ids = [101] + input_ids + [102] + [0]* padding_len
        padded_token_type_ids = [0] + token_type_ids + [0] + [0]* padding_len
        padded_attention_mask = [1] + attention_mask + [1] + [0]* padding_len
        padded_labels = [0] + labels + [0] + [0]* padding_len
        return padded_input_ids, padded_token_type_ids, padded_attention_mask, padded_labels


train_set = NER_Dataset(train_dataset)
validation_set = NER_Dataset(validation_dataset)
test_set = NER_Dataset(test_dataset)

batch_size = 16

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, pin_memory=True)
validation_loader = DataLoader(validation_set, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True, pin_memory=True)


In [7]:
# Obtain pred_labels and true_labels

def remove_zero(a_list):
    return [x for x in a_list if x != 0]

def most_frequent(a_list):
    return max(set(a_list), key=a_list.count)

def multi_pop(a_list, times):
    return a_list[times:]

def obtain_pred_true_labels(data, output_logits, batch_size=batch_size):
    pred_labels_list, true_labels_list = [], []
    pred_labels, true_labels = torch.argmax(output_logits, dim=2).tolist(), data[3].tolist()
    num_of_tokens = [remove_zero(x) for x in data[4].tolist()]

    for i in range(batch_size):
        one_case_pred_label, one_case_true_label = pred_labels[i], true_labels[i]

        # pop the first token [CLS]
        one_case_pred_label.pop(0)
        one_case_true_label.pop(0)

        tem_pred_labels, tem_true_labels = [], []

        for j in num_of_tokens[i]:
            # parse pred label
            a_list = one_case_pred_label[:j]
            tem_pred_labels.append(most_frequent(a_list))
            one_case_pred_label = multi_pop(one_case_pred_label, j)

            # parse true label
            a_list = one_case_true_label[:j]
            tem_true_labels.append(most_frequent(a_list))
            one_case_true_label = multi_pop(one_case_true_label, j)

        pred_labels_list.append(tem_pred_labels)
        true_labels_list.append(tem_true_labels)
        
    return pred_labels_list, true_labels_list


# transform predictive labels and true labels to ner_tags with
# ner_tags = {0:"O", 1:"B-PER", 2:"I-PER", 3:"B-ORG", 4:"I-ORG", 5:"B-LOC", 6:"I-LOC"}
def transform_2_ner_tags(a_list):
    return([ner_tags[x] for x in a_list])


In [8]:
# Training Model

num_epoch = 10
evaluate_test = True
logging_step = 100
learning_rate = 1e-4
best_f1 = 0
optimizer = AdamW(model.parameters(), lr=learning_rate)

model.train()

print('Progress Bar of Epoch')
for epoch in tqdm(range(num_epoch)):
    # initial setting
    print('- '* 30)
    print('Model Training ...')
    train_pred_labels_list, train_true_labels_list = list(), list()
    validation_pred_labels_list, validation_true_labels_list = list(), list()
    test_pred_labels_list, test_true_labels_list = list(), list()
    step = 1
    train_loss = 0
    
    for data in tqdm(train_loader):
        data = [i.to(device) for i in data]
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], labels=data[3])
        output_logits = output.logits
        
        # obtain predictive labels and true labels
        pred_labels_list, true_labels_list = obtain_pred_true_labels(data=data, output_logits=output_logits, batch_size=batch_size)
        
        # transform predictive labels and true labels to ner_tags
        pred_labels_list_2_ner_tags = [transform_2_ner_tags(x) for x in pred_labels_list]
        true_labels_list_2_ner_tags = [transform_2_ner_tags(x) for x in true_labels_list]
        
        # append
        train_pred_labels_list += pred_labels_list_2_ner_tags
        train_true_labels_list += true_labels_list_2_ner_tags
        
        # accumulate loss
        train_loss += output.loss
        
        # backward
        output.loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        step += 1
        
        if step % logging_step == 0:
            train_f1_score = f1_score(train_true_labels_list, train_pred_labels_list)
            print(f"Epoch {epoch + 1} | Step {step:4d} | loss = {train_loss.item() / logging_step:.3f} | train_f1_score = {100* train_f1_score:.2f}")
            train_loss = 0
            train_pred_labels_list, train_true_labels_list = list(), list()
        
        
    # Evaluate validation dataset
    print('Evaluate validation dataset ...')
    model.eval()
    with torch.no_grad():
        for data in tqdm(validation_loader):
            data = [i.to(device) for i in data]
            output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], labels=data[3])
            output_logits = output.logits

            # obtain predictive labels and true labels
            pred_labels_list, true_labels_list = obtain_pred_true_labels(data=data, output_logits=output_logits, batch_size=batch_size)

            # transform predictive labels and true labels to ner_tags
            pred_labels_list_2_ner_tags = [transform_2_ner_tags(x) for x in pred_labels_list]
            true_labels_list_2_ner_tags = [transform_2_ner_tags(x) for x in true_labels_list]

            # append
            validation_pred_labels_list += pred_labels_list_2_ner_tags
            validation_true_labels_list += true_labels_list_2_ner_tags

    print('- '* 30)
    validation_f1_score = f1_score(validation_true_labels_list, validation_pred_labels_list)
    print(f"Validation | Epoch {epoch + 1} | validation_f1_score = {100* validation_f1_score:.2f}")
    print('- '* 30)
    validation_pred_labels_list, validation_true_labels_list = list(), list()
    model.train()
    
    
    # save model if the f1 score in validation dataset greater than previous f1 score
    if validation_f1_score > best_f1:
        best_f1 = validation_f1_score
        print('Saving Model ...')
        if 'saved_model' not in os.listdir():
            os.mkdir('saved_model')
        # saved model can be re-loaded using { model = pickle.load(open('saved_model/model.pkl', 'rb')) }
        pickle.dump(model, open('saved_model/model.pkl', 'wb'))
        
        
    # Evaluate test dataset
    if evaluate_test:
        print('Evaluate test dataset ...')
        model.eval()
        with torch.no_grad():
            for data in tqdm(test_loader):
                data = [i.to(device) for i in data]
                output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], labels=data[3])
                output_logits = output.logits
                
                # obtain predictive labels and true labels
                pred_labels_list, true_labels_list = obtain_pred_true_labels(data=data, output_logits=output_logits, batch_size=batch_size)
                
                # transform predictive labels and true labels to ner_tags
                pred_labels_list_2_ner_tags = [transform_2_ner_tags(x) for x in pred_labels_list]
                true_labels_list_2_ner_tags = [transform_2_ner_tags(x) for x in true_labels_list]
                
                # append
                test_pred_labels_list += pred_labels_list_2_ner_tags
                test_true_labels_list += true_labels_list_2_ner_tags

        print('- '* 30)
        test_f1_score = f1_score(test_true_labels_list, test_pred_labels_list)
        print(f"Test | Epoch {epoch + 1} | test_f1_score = {100* test_f1_score:.2f}")
        print('- '* 30)
        test_pred_labels_list, test_true_labels_list = list(), list()
        model.train()
    


Progress Bar of Epoch


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Model Training ...


Epoch 1 | Step  100 | loss = 0.618 | train_f1_score = 47.15
Epoch 1 | Step  200 | loss = 0.384 | train_f1_score = 64.87
Epoch 1 | Step  300 | loss = 0.371 | train_f1_score = 67.40
Epoch 1 | Step  400 | loss = 0.306 | train_f1_score = 72.24
Epoch 1 | Step  500 | loss = 0.320 | train_f1_score = 72.94
Epoch 1 | Step  600 | loss = 0.308 | train_f1_score = 73.42
Epoch 1 | Step  700 | loss = 0.296 | train_f1_score = 73.08
Epoch 1 | Step  800 | loss = 0.263 | train_f1_score = 75.36
Epoch 1 | Step  900 | loss = 0.295 | train_f1_score = 73.78
Epoch 1 | Step 1000 | loss = 0.274 | train_f1_score = 74.71
Epoch 1 | Step 1100 | loss = 0.297 | train_f1_score = 71.73
Epoch 1 | Step 1200 | loss = 0.264 | train_f1_score = 76.01

Evaluate validation dataset ...



- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Validation | Epoch 1 | validation_f1_score = 76.97
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Evaluate test dataset ...



- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Test | Epoch 1 | test_f1_score = 76.35
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Model Training ...


...


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Validation | Epoch 9 | validation_f1_score = 76.24
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Evaluate test dataset ...



- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Test | Epoch 9 | test_f1_score = 75.81
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Model Training ...


Epoch 10 | Step  100 | loss = 0.038 | train_f1_score = 94.88
Epoch 10 | Step  200 | loss = 0.035 | train_f1_score = 95.74
Epoch 10 | Step  300 | loss = 0.047 | train_f1_score = 94.58
Epoch 10 | Step  400 | loss = 0.041 | train_f1_score = 94.67
Epoch 10 | Step  500 | loss = 0.118 | train_f1_score = 89.20
Epoch 10 | Step  600 | loss = 0.074 | train_f1_score = 90.55
Epoch 10 | Step  700 | loss = 0.069 | train_f1_score = 93.42
Epoch 10 | Step  800 | loss = 0.047 | train_f1_score = 94.16
Epoch 10 | Step  900 | loss = 0.048 | train_f1_score = 93.99
Epoch 10 | Step 1000 | loss = 0.051 | train_f1_score = 92.12
Epoch 10 | Step 1100 | loss = 0.054 | train_f1_score = 94.50
Epoch 10 | Step 1200 | loss = 0.043 | train_f1_score = 94.47

Evaluate validation dataset ...



- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Validation | Epoch 10 | validation_f1_score = 77.54
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Evaluate test dataset ...



- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Test | Epoch 10 | test_f1_score = 77.02
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Saving Model ...
